# データ前処理

このノートブックは、Tennis Visionアプリケーションのデータ前処理タスク専用です。生データをロードし、クリーニングし、トレーニング用に準備します。

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import subprocess
import shutil

# パスを定義
raw_data_path = '../data/raw/'
processed_data_path = '../data/processed/'

# 生の動画ファイルをロードする関数
def load_raw_videos(path):
    videos = []
    for filename in os.listdir(path):
        if filename.endswith(('.mp4', '.avi', '.mov', '.mkv')): # H.265で一般的な拡張子を追加
            video_path = os.path.join(path, filename)
            videos.append(video_path)
    return videos

# 生の動画をロード
raw_videos = load_raw_videos(raw_data_path)
print(f'{raw_data_path} から {len(raw_videos)} 本の動画をロードしました')

../data/raw/ から 1 本の動画をロードしました


In [ ]:
# 動画フレームを前処理する関数
def preprocess_video(video_path):
    video_name = os.path.basename(video_path).split('.')[0]
    temp_frame_output_folder = os.path.join(processed_data_path, f'{video_name}_temp_frames')
    os.makedirs(temp_frame_output_folder, exist_ok=True)

    # ffmpegコマンドでフレームを抽出
    # -i: 入力ファイル
    # -vf fps=60: 1秒あたり60フレームを抽出 (必要に応じて変更)
    # %06d.png: フレームを6桁の連番のPNGファイルとして保存 (フレーム数が多い場合に対応)
    command = [
        'ffmpeg',
        '-i', video_path,
        '-vf', 'fps=60', # 例として60fpsで抽出。必要に応じて調整してください。
        os.path.join(temp_frame_output_folder, '%06d.png') # フレーム番号の桁数を増やすことを検討
    ]
    
    try:
        subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    except subprocess.CalledProcessError as e:
        print(f"ffmpeg の実行中にエラーが発生しました ({video_name}): {e.stderr.decode()}")
        shutil.rmtree(temp_frame_output_folder) # エラー時は一時フォルダを削除
        return []

    frames_from_video = []
    frame_files = sorted(os.listdir(temp_frame_output_folder))
    for frame_file in frame_files:
        if frame_file.endswith('.png'):
            frame_image_path = os.path.join(temp_frame_output_folder, frame_file)
            frame = cv2.imread(frame_image_path)
            if frame is not None:
                frames_from_video.append(frame)
            else:
                print(f"フレームの読み込みに失敗しました: {frame_image_path}")
    
    # 一時フォルダを削除
    shutil.rmtree(temp_frame_output_folder)
    
    return frames_from_video

# すべての動画を前処理し、処理済みフレームを単一の.npyファイルに保存
all_processed_frames = []
for video_path in raw_videos: # 変数名を video から video_path に変更
    print(f'{os.path.basename(video_path)} のフレームを処理中...')
    frames = preprocess_video(video_path)
    if frames: # フレームが正常に抽出された場合のみ追加
        all_processed_frames.extend(frames)
        print(f'{os.path.basename(video_path)} から {len(frames)} フレームをリストに追加しました。')
    else:
        print(f'{os.path.basename(video_path)} のフレーム処理に失敗、またはフレームがありませんでした。')

if all_processed_frames:
    # 全フレームをNumPy配列に変換（メモリ使用量に注意）
    all_frames_array = np.array(all_processed_frames)
    output_npy_path = os.path.join(processed_data_path, 'all_video_frames.npy')
    np.save(output_npy_path, all_frames_array)
    print(f'すべての動画の全 {len(all_processed_frames)} フレームを {output_npy_path} に保存しました。')
    print(f'保存されたNPY配列の形状: {all_frames_array.shape}')
else:
    print('処理できるフレームがありませんでした。npyファイルは保存されません。')


output のフレームを処理して保存しました


### .npy ファイルについて

`.npy` ファイルは、NumPyライブラリで使用される、単一のNumPy配列をディスクに保存するための標準的なバイナリファイル形式です。

このノートブックでは、`preprocess_video` 関数によって各動画から抽出され、前処理されたフレームがすべて収集され、**単一の大きな `.npy` ファイル** (例: `all_video_frames.npy`) として保存されます。このファイルには、処理されたすべての動画の全フレームが順番に格納されています。これにより、後の機械学習モデルのトレーニングフェーズで、これらのフレームデータを一括で、かつ効率的に読み込むことができます。

主な利点は以下の通りです。
- **効率性**: バイナリ形式であるため、テキストファイルよりも小さく、読み書きが高速です。
- **利便性**: NumPy配列の構造（形状、データ型など）をそのまま保持して保存・復元できます。
- **移植性**: NumPyがインストールされていれば、異なるシステム間でもファイルを共有できます。

**注意点**:
- すべてのフレームを単一の `.npy` ファイルにまとめると、ファイルサイズが非常に大きくなる可能性があります。
- トレーニング時には、この大きな `.npy` ファイルから特定のフレーム（またはそのインデックス）を効率的に読み込むためのカスタムデータローダーの実装が重要になります。各フレームが元々どの動画の何番目のフレームだったかの情報を別途管理する必要があるかもしれません（例: `train.txt` やメタデータファイル）。

### 処理後のフレームの画質について

`preprocess_video` 関数で処理された後のフレームの画質は、主に以下の要因に依存します。

1.  **元の動画の画質**:
    最も基本的な要因です。元の動画の解像度、ビットレート、圧縮方式などが、抽出されるフレームの画質のベースとなります。

2.  **`ffmpeg` によるフレーム抽出**:
    *   現在のスクリプトでは、`ffmpeg` を使用して動画からフレームを抽出し、一時的に `.png` ファイルとして保存しています。PNGは可逆圧縮形式であるため、この段階で `ffmpeg` がデコードしたフレームからの画質劣化はほとんどありません。
    *   抽出時の解像度は、`ffmpeg` のコマンドで `-vf scale=width:height` のようなフィルタを指定しない限り、元の動画の解像度が維持されます。現在のスクリプトでは解像度変更は行っていません。
    *   フレームレートは `fps=60` (または指定した値) に設定されますが、これは時間的なサンプリングレートであり、個々のフレームの空間的な画質（解像度や鮮明さ）に直接影響するものではありません。

3.  **`cv2.imread` による読み込みと `.npy` 形式での保存**:
    *   一時保存された `.png` フレームは `cv2.imread` によって読み込まれます。OpenCVが正しく画像を読み込める限り、ここでの画質劣化も最小限です。
    *   その後、フレームデータ（NumPy配列）は `.npy` ファイルとして保存されます。`.npy` 形式はデータをバイナリでそのまま保存するため、この保存処理自体が画質を劣化させることはありません。

**結論として、この前処理スクリプトで処理されたフレームの画質は、元の動画の画質に非常に近いものとなることが期待されます。** スクリプト内では、解像度の変更や画質を意図的に低下させるような非可逆圧縮処理は行われていません。ただし、元の動画が非常に高色深度である場合など、標準的な画像処理ライブラリ（OpenCVが内部的に扱うデータ型、通常は8ビット/チャネル）で表現しきれない微細な情報が失われる可能性はゼロではありませんが、一般的な動画であれば問題になることは稀です。

## まとめ

このノートブックでは、生の動画データをロードし、ffmpegを使用してフレームを抽出し、モデルトレーニングでさらに使用するために、**すべての動画から抽出した全フレームを単一の `.npy` ファイルとして**保存しました。